# Plotting Rocket Launch Sites with Folium

In [1]:
import pandas as pd
import wikipedia as wp
import numpy as np

## Scrape data from Wikipedia

Out of all the internet sources, only wikipedia has this convenient table that's ripe for scraping

In [2]:
html = wp.page("List of rocket launch sites").html().encode("UTF-32")

## Africa

In [3]:
Africa = pd.read_html(html, header=0)[2]
Africa.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,Algeria,"Hammaguira French Special Weapons Test Centre,...",31°05′58″N 2°50′09″W﻿ / ﻿31.09951°N 2.83581°W﻿...,1947–1967,NaN,18 000 kg,Orbital,Operated by France.[1]
1,Algeria,Reggane,26°43′08″N 0°16′37″E﻿ / ﻿26.71895°N 0.27691°E﻿...,1961–1965,10.0,NaN,NaN,NaN
2,Congo (Zaire),"Shaba North, Kapani Tonneo OTRAG Launch Center",7°55′33″S 28°31′40″E﻿ / ﻿7.92587°S 28.52766°E﻿...,1977–1978,3.0,NaN,<50 km,German OTRAG rockets.[2]
3,Egypt,Jabal Hamzah ballistic missile test and launch...,30°07′32.7″N 30°36′18.5″E﻿ / ﻿30.125750°N 30.6...,Late 1950s–present,6.0,NaN,NaN,Al Zafir and Al Kahir SRBMs testing[3][4]
4,Kenya,"Broglio Space Centre (San Marco), Malindi",2°56′27″S 40°12′48″E﻿ / ﻿2.94080°S 40.21340°E﻿...,1964–1988,27.0,20 000 kg,Orbital,"Scout rockets, operated by ASI and Sapienza Un..."


We'll plot each site with the decimal form of the coordinates. Data cleaning has to be performed in order to create a column for the latitudes and the longitudes.

This involves splitting each row value as list through recurrent strings while recalling only the first value of the list.

In [4]:
Africa['Coordinates'][0]

'31°05′58″N 2°50′09″W\ufeff / \ufeff31.09951°N 2.83581°W\ufeff / 31.09951; -2.83581\ufeff (Hammaguir)'

In [5]:
Africa['Coordinates'][0].split()

['31°05′58″N',
 '2°50′09″W\ufeff',
 '/',
 '\ufeff31.09951°N',
 '2.83581°W\ufeff',
 '/',
 '31.09951;',
 '-2.83581\ufeff',
 '(Hammaguir)']

In [6]:
Africa['Coordinates'][0].split('\ufeff / ')

['31°05′58″N 2°50′09″W',
 '\ufeff31.09951°N 2.83581°W',
 '31.09951; -2.83581\ufeff (Hammaguir)']

In [7]:
Africa['Location'] = Africa['Location'].apply(lambda name: name.split(',')[0])
Africa['Location']

0        Hammaguira French Special Weapons Test Centre
1                                              Reggane
2                                          Shaba North
3    Jabal Hamzah ballistic missile test and launch...
4                     Broglio Space Centre (San Marco)
5                                                Sabha
6                                           Nouadhibou
7                   Overberg South African Test Centre
Name: Location, dtype: object

### For popups

Clicking each plotted location would produce these popups. In this notebook, the popups would be the location and the deg hr min form of the coordinates.

In [8]:
Africa['Coordinates'][0].split('\ufeff / ')[0]

'31°05′58″N 2°50′09″W'

### Africa: Latitude

We'll create a new column called 'Loc_string' as the source of the latitude and longitude data.

In [9]:
Africa['Loc_string'] = Africa['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])
Africa['Loc_string']

0             31.09951; -2.83581﻿ (Hammaguir)
1                26.71895; 0.27691﻿ (Reggane)
2         -7.92587; 28.52766﻿ (Kapani Tonneo)
3        30.125750; 30.605139﻿ (Jabal Hamzah)
4    -2.94080; 40.21340﻿ (San Marco platform)
5                 26.99392; 14.46425﻿ (Sabha)
6           20.92856; -17.03153﻿ (Nouadhibou)
7             -34.60265; 20.30248﻿ (Overberg)
Name: Loc_string, dtype: object

In [10]:
Africa['Loc_string'].apply(lambda name: name.split(';')[0])

0     31.09951
1     26.71895
2     -7.92587
3    30.125750
4     -2.94080
5     26.99392
6     20.92856
7    -34.60265
Name: Loc_string, dtype: object

In [11]:
Africa['Latitude'] = Africa['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)
Africa['Latitude']

0    31.09951
1    26.71895
2    -7.92587
3    30.12575
4    -2.94080
5    26.99392
6    20.92856
7   -34.60265
Name: Latitude, dtype: float64

### Africa: Longitude

In [12]:
Africa['Loc_string'][0].split('\ufeff ')

['31.09951; -2.83581', '(Hammaguir)']

In [13]:
Africa['Loc_string']

0             31.09951; -2.83581﻿ (Hammaguir)
1                26.71895; 0.27691﻿ (Reggane)
2         -7.92587; 28.52766﻿ (Kapani Tonneo)
3        30.125750; 30.605139﻿ (Jabal Hamzah)
4    -2.94080; 40.21340﻿ (San Marco platform)
5                 26.99392; 14.46425﻿ (Sabha)
6           20.92856; -17.03153﻿ (Nouadhibou)
7             -34.60265; 20.30248﻿ (Overberg)
Name: Loc_string, dtype: object

In [14]:
Africa['Loc_string'].apply(lambda name: name.split()[5-4])[0]

'-2.83581\ufeff'

In [15]:
Africa['Longitude'] = Africa['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)
Africa['Longitude']

0    -2.835810
1     0.276910
2    28.527660
3    30.605139
4    40.213400
5    14.464250
6   -17.031530
7    20.302480
Name: Longitude, dtype: float64

In [16]:
Africa

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes,Loc_string,Latitude,Longitude
0,Algeria,Hammaguira French Special Weapons Test Centre,31°05′58″N 2°50′09″W﻿ / ﻿31.09951°N 2.83581°W﻿...,1947–1967,NaN,18 000 kg,Orbital,Operated by France.[1],31.09951; -2.83581﻿ (Hammaguir),31.09951,-2.835810
1,Algeria,Reggane,26°43′08″N 0°16′37″E﻿ / ﻿26.71895°N 0.27691°E﻿...,1961–1965,10.0,NaN,NaN,NaN,26.71895; 0.27691﻿ (Reggane),26.71895,0.276910
2,Congo (Zaire),Shaba North,7°55′33″S 28°31′40″E﻿ / ﻿7.92587°S 28.52766°E﻿...,1977–1978,3.0,NaN,<50 km,German OTRAG rockets.[2],-7.92587; 28.52766﻿ (Kapani Tonneo),-7.92587,28.527660
3,Egypt,Jabal Hamzah ballistic missile test and launch...,30°07′32.7″N 30°36′18.5″E﻿ / ﻿30.125750°N 30.6...,Late 1950s–present,6.0,NaN,NaN,Al Zafir and Al Kahir SRBMs testing[3][4],30.125750; 30.605139﻿ (Jabal Hamzah),30.12575,30.605139
4,Kenya,Broglio Space Centre (San Marco),2°56′27″S 40°12′48″E﻿ / ﻿2.94080°S 40.21340°E﻿...,1964–1988,27.0,20 000 kg,Orbital,"Scout rockets, operated by ASI and Sapienza Un...",-2.94080; 40.21340﻿ (San Marco platform),-2.94080,40.213400
5,Libya,Sabha,26°59′38″N 14°27′51″E﻿ / ﻿26.99392°N 14.46425°...,1981–1982,NaN,NaN,50 km,German OTRAG rockets.[6],26.99392; 14.46425﻿ (Sabha),26.99392,14.464250
6,Mauritania,Nouadhibou,20°55′43″N 17°01′54″W﻿ / ﻿20.92856°N 17.03153°...,1973–1973,1.0,NaN,NaN,During a solar eclipse,20.92856; -17.03153﻿ (Nouadhibou),20.92856,-17.031530
7,South Africa,Overberg South African Test Centre,34°36′10″S 20°18′09″E﻿ / ﻿34.60265°S 20.30248°...,1989–1990,NaN,NaN,NaN,Launched test mission rockets only.[7],-34.60265; 20.30248﻿ (Overberg),-34.60265,20.302480


## Plot with Folium

Making folium maps begin with producing a blank map. This blank map would then be updated throughout as we go by.

In [17]:
import folium
from folium.plugins import MarkerCluster

In [18]:
Map = folium.Map(location=[0,0], height=550, width=950, tiles="OpenStreetMap", 
                   zoom_start=1.5)
Map

## Update map with African markplots

Olympic colors would be used. Black for Africa, Yellow for Asia, Blue for Europe, Red for the New World/Americas and Green for Oceania. 

In order to plot the pop-ups with ' and &, we set parse_html=True.

In [19]:
for i in np.arange(len(Africa)):
    lat = Africa['Latitude'][i]
    lon = Africa['Longitude'][i]
    folium.CircleMarker([Africa['Latitude'][i], Africa['Longitude'][i]], 
                  radius=2.5, color='black', fill=True, fill_color='black', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Africa['Location'].values[i], 
                         Africa['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)

Map                  

We save each update to an html file.

In [20]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## Asia

In [21]:
Asia = pd.read_html(html, header=0)[3]
Asia.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,China,"Base 603, Shijiedu, Guangde",30°56′15″N 119°12′21″E﻿ / ﻿30.93743°N 119.2057...,1960–1966,NaN,1 000 kg,<60 km,NaN
1,China,Jiuquan Satellite Launch Center,41°07′05″N 100°27′48″E﻿ / ﻿41.11803°N 100.4633...,1970–,NaN,464 000 kg,Orbital,Human spaceflight[8]
2,China,Taiyuan Satellite Launch Center,39°08′36″N 111°58′03″E﻿ / ﻿39.14321°N 111.9674...,1980–,NaN,NaN,Orbital,Polar satellites[9]
3,China,Xichang Satellite Launch Center,28°14′47″N 102°01′41″E﻿ / ﻿28.24646°N 102.0281...,1984–,NaN,NaN,Lunar,"Geo-synchronous satellites, lunar probes.[10][11]"
4,China,Wenchang Satellite Launch Center,19°36′52.17″N 110°57′4.08″E﻿ / ﻿19.6144917°N 1...,2016–,NaN,879 000 kg,Orbital,New site on Hainan Island with pads for Long M...


In [22]:
Asia['Loc_string'] = Asia['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])

In [23]:
Asia['Location']

0                           Base 603, Shijiedu, Guangde
1                       Jiuquan Satellite Launch Center
2                       Taiyuan Satellite Launch Center
3                       Xichang Satellite Launch Center
4                      Wenchang Satellite Launch Center
5                                                Jingyu
6     Vikram Sarabhai Space Centre, Thiruvananthapur...
7     Satish Dhawan Space Centre (Sriharikota), Andh...
8                                    Abdul Kalam Island
9          Stasiun Peluncuran Roket, Pameungpeuk, Garut
10                                     Qom Space Center
11                               Emamshahr Space Center
12                                     Semnan spaceport
13                                 Al-Anbar Test Center
14                             Palmachim Air Force Base
15                                   Akita Rocket Range
16                               Uchinoura Space Center
17         Tanegashima Space Center, Tanegashima

In [24]:
Asia['Location'] = Asia['Location'].apply(lambda name: name.split('[')[0])

We could remove the unwanted [citation needed] part if we split through '['.

In [25]:
Asia['Location']

0                           Base 603, Shijiedu, Guangde
1                       Jiuquan Satellite Launch Center
2                       Taiyuan Satellite Launch Center
3                       Xichang Satellite Launch Center
4                      Wenchang Satellite Launch Center
5                                                Jingyu
6     Vikram Sarabhai Space Centre, Thiruvananthapur...
7     Satish Dhawan Space Centre (Sriharikota), Andh...
8                                    Abdul Kalam Island
9          Stasiun Peluncuran Roket, Pameungpeuk, Garut
10                                     Qom Space Center
11                               Emamshahr Space Center
12                                     Semnan spaceport
13                                 Al-Anbar Test Center
14                             Palmachim Air Force Base
15                                   Akita Rocket Range
16                               Uchinoura Space Center
17         Tanegashima Space Center, Tanegashima

### Asia: Latitude

In [26]:
Asia['Latitude'] = Asia['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)

### Asia: Longitude

In [27]:
Asia['Longitude'] = Asia['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)

From this point onward, the steps in cleaning per continent would be repeated.

## Update map with Asian markplots

Folium doesn't have a string, 'yellow' for color. I used 'orange' instead.

In [28]:
for i in np.arange(len(Asia)):
    folium.CircleMarker([Asia['Latitude'][i], Asia['Longitude'][i]], 
                  radius=2.5, color='orange', fill=True, fill_color='orange', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Asia['Location'].values[i], 
                         Asia['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)
Map   

In [29]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## Europe

In [30]:
Europe = pd.read_html(html, header=0)[4]
Europe.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,France,Ile du Levant,43°02′42″N 6°28′44″E﻿ / ﻿43.04507°N 6.47887°E﻿...,1948–[citation needed],NaN,NaN,NaN,NaN
1,Germany,"Rocket Launch Site Berlin, Berlin-Tegel",52°21′00″N 13°12′36″E﻿ / ﻿52.35000°N 13.21000°...,1930–1933,NaN,NaN,4 km,NaN
2,Germany,Peenemünde/Greifswalder Oie,54°08′35″N 13°47′38″E﻿ / ﻿54.14300°N 13.79400°...,1942–1945,>3000,12 500 kg,>100 km,"V-2 rockets during World War II, first rocket ..."
3,Germany,Cuxhaven,53°50′56″N 8°35′30″E﻿ / ﻿53.84884°N 8.59154°E﻿...,1945–1964,NaN,NaN,NaN,NaN
4,Germany,"Hespenbusch, Großenkneten",52°56′20″N 8°18′45″E﻿ / ﻿52.939002°N 8.312515°...,1952–1957,NaN,NaN,<10 km,NaN


In [31]:
Europe['Loc_string'] = Europe['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])
Europe['Location'] = Europe['Location'].apply(lambda name: name.split('[')[0])
Europe['Latitude'] = Europe['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)
Europe['Longitude'] = Europe['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)

In [32]:
for i in np.arange(len(Europe)):
    folium.CircleMarker([Europe['Latitude'][i], Europe['Longitude'][i]], 
                  radius=2.5, color='blue', fill=True, fill_color='blue', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Europe['Location'].values[i], 
                         Europe['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)
Map    

In [33]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## North America

In [34]:
N_America = pd.read_html(html, header=0)[5]
N_America.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,Canada,"Churchill Rocket Research Range, Manitoba",58°44′03″N 93°49′13″W﻿ / ﻿58.73430°N 93.82030°...,1954–1998,>3500,NaN,NaN,Canadian Army.[41]
1,Canada,"Resolute Bay, Nunavut",74°41′13″N 94°53′46″W﻿ / ﻿74.6870°N 94.8962°W﻿...,1966–1971,17,NaN,NaN,National Research Council Canada.[42]
2,Canada,Hall Beach,68°46′34″N 81°14′36″W﻿ / ﻿68.77607°N 81.24346°...,1971–1971,7,NaN,270 km,NaN
3,Canada,Southend,56°20′N 103°14′W﻿ / ﻿56.333°N 103.233°W﻿ / 56....,1980–1980,2,1 200 kg,NaN,NaN
4,Greenland (Denmark),Thule Air Base,76°25′26″N 68°17′37″W﻿ / ﻿76.4240°N 68.2936°W﻿...,1964–1980,NaN,NaN,NaN,US Air Force[43]


In [35]:
N_America['Loc_string'] = N_America['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])
N_America['Location'] = N_America['Location'].apply(lambda name: name.split('[')[0])
N_America['Latitude'] = N_America['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)
N_America['Longitude'] = N_America['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)

In [36]:
for i in np.arange(len(N_America)):
    folium.CircleMarker([N_America['Latitude'][i], N_America['Longitude'][i]], 
                  radius=2.5, color='red', fill=True, fill_color='red', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(N_America['Location'].values[i], 
                         N_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)
Map

In [37]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## South America

In [38]:
S_America = pd.read_html(html, header=0)[7]
S_America.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,Argentina,Puerto Belgrano,38°57′46″S 61°42′54″W﻿ / ﻿38.9628°S 61.715°W﻿ ...,TBA,NaN,NaN,NaN,Launch pad for Tronador II under construction ...
1,Argentina,Punta Indio,35°31′26″S 57°11′02″W﻿ / ﻿35.523889°S 57.18388...,2014–,3,7 076 kg [63],2.2 km,Civilian Launch Test Site (CONAE)
2,Argentina,CELPA,30°07′39″S 66°20′27″W﻿ / ﻿30.12737°S 66.34082°...,1962–1974 / 2011–,100 (approximately),933 kg,550 km,Military Launch Test Site (CITIDEF)
3,Argentina,Serrezuela,30°38′00″S 65°23′00″W﻿ / ﻿30.6333°S 65.3833°W﻿...,2009,1,500 kg,40 km,Military test (Gradicom I)
4,Argentina,Las Palmas,27°05′43″S 58°45′13″W﻿ / ﻿27.09531°S 58.75352°...,1966–1966,2,3 400 kg,270 km,"During a solar eclipse, with Titus rockets."


In [39]:
S_America['Loc_string'] = S_America['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])
S_America['Location'] = S_America['Location'].apply(lambda name: name.split('[')[0])
S_America['Latitude'] = S_America['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)
S_America['Longitude'] = S_America['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)

In [40]:
for i in np.arange(len(S_America)):
    folium.CircleMarker([S_America['Latitude'][i], S_America['Longitude'][i]], 
                  radius=2.5, color='red', fill=True, fill_color='red', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(S_America['Location'].values[i], 
                         S_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)
Map    

In [41]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## Oceania

In [42]:
Oceania = pd.read_html(html, header=0)[8]
Oceania.head()

,Country,Location,Coordinates,Operational date,Number of rocket launches,Heaviest rocket launched,Highest achieved altitude,Notes
0,Australia,Woomera Test Range,30°57′31″S 136°30′13″E﻿ / ﻿30.95875°S 136.5036...,1950s–,NaN,28 000 kg,Orbital,British/Australian government research facilit...
1,Australia,Carnarvon,24°29′08″S 113°24′31″E﻿ / ﻿24.48564°S 113.4086...,1964–1965,12.0,NaN,120 km,NaN
2,Australia,Lancelin,30°58′59″S 115°19′04″E﻿ / ﻿30.98309°S 115.3177...,1974–1974,2.0,NaN,NaN,During a total solar eclipse
3,Marshall Islands,"Reagan Test Site, Omelek Island, Kwajalein Atoll",9°02′53″N 167°44′35″E﻿ / ﻿9.048167°N 167.74308...,1950s-,NaN,39 000 kg,Orbital,US-controlled ICBM base converted for satellit...
4,New Zealand,Birdling's Flat,43°49′01″S 172°40′59″E﻿ / ﻿43.81700°S 172.6830...,NaN,NaN,NaN,NaN,"Formerly proposed launch site for Rocket Lab, ..."


In [43]:
Oceania['Loc_string'] = Oceania['Coordinates'].apply(lambda name: name.split('\ufeff / ')[-1])
Oceania['Location'] = Oceania['Location'].apply(lambda name: name.split('[')[0])
Oceania['Latitude'] = Oceania['Loc_string'].apply(lambda name: name.split(';')[0]).astype(float)
Oceania['Longitude'] = Oceania['Loc_string'].apply(lambda name: name.split()[1].rstrip('\ufeff')).astype(float)

In [44]:
for i in np.arange(len(Oceania)):
    folium.CircleMarker([Oceania['Latitude'][i], Oceania['Longitude'][i]], 
                  radius=2.5, color='green', fill=True, fill_color='green', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Oceania['Location'][i], 
                         Oceania['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map)
Map

In [45]:
fn_out = "map_rocket_launch_sites.html"
Map.save(fn_out)

## Plot as Cluster Map

Try with tiles="Stamen Terrain".

In [46]:
Map_cluster = folium.Map(location=[0,0], height=550, width=950, tiles="Stamen Terrain",
                   zoom_start=1.5)
Map_cluster

In [47]:
marker_cluster = MarkerCluster().add_to(Map_cluster)

In [48]:
for i in np.arange(len(Africa)):
    lat = Africa['Latitude'][i]
    lon = Africa['Longitude'][i]
    folium.CircleMarker([Africa['Latitude'][i], Africa['Longitude'][i]], 
                  radius=2.5, color='black', fill=True, fill_color='black', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Africa['Location'].values[i], 
                         Africa['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)

for i in np.arange(len(Asia)):
    folium.CircleMarker([Asia['Latitude'][i], Asia['Longitude'][i]], 
                  radius=2.5, color='orange', fill=True, fill_color='orange', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Asia['Location'].values[i], 
                         Asia['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)

for i in np.arange(len(Europe)):
    folium.CircleMarker([Europe['Latitude'][i], Europe['Longitude'][i]], 
                  radius=2.5, color='blue', fill=True, fill_color='blue', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Europe['Location'].values[i], 
                         Europe['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)

for i in np.arange(len(N_America)):
    folium.CircleMarker([N_America['Latitude'][i], N_America['Longitude'][i]], 
                  radius=2.5, color='red', fill=True,  fill_color='red', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(N_America['Location'].values[i], 
                         N_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)
    
for i in np.arange(len(S_America)):
    folium.CircleMarker([S_America['Latitude'][i], S_America['Longitude'][i]], 
                  radius=2.5, color='red', fill=True,  fill_color='red', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(S_America['Location'].values[i], 
                         S_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)

for i in np.arange(len(Oceania)):
    folium.CircleMarker([Oceania['Latitude'][i], Oceania['Longitude'][i]], 
                  radius=2.5, color='green', fill=True,  fill_color='green', fill_opacity=0.5,
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Oceania['Location'][i], 
                         Oceania['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(marker_cluster)

Map_cluster               

In [49]:
fn_out2 = "cluster_map_rocket_launch_sites.html"
Map_cluster.save(fn_out2)

## Plot with Font Awesome Rocket Icon

In [50]:
Map_icon = folium.Map(location=[0,0], height=550, width=950, tiles="OpenStreetMap", 
                   zoom_start=1.5)
Map_icon

In [51]:
for i in np.arange(len(Africa)):
    lat = Africa['Latitude'][i]
    lon = Africa['Longitude'][i]
    folium.Marker([Africa['Latitude'][i], Africa['Longitude'][i]], 
                  icon=folium.Icon(color='black',icon='rocket', prefix='fa'),
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Africa['Location'].values[i], 
                         Africa['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
for i in np.arange(len(Asia)):
    folium.Marker([Asia['Latitude'][i], Asia['Longitude'][i]], 
                  icon=folium.Icon(color='orange',icon='rocket', prefix='fa'),
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Asia['Location'].values[i], 
                         Asia['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
for i in np.arange(len(Europe)):
    folium.Marker([Europe['Latitude'][i], Europe['Longitude'][i]], 
                  icon=folium.Icon(color='blue',icon='rocket', prefix='fa'), 
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Europe['Location'].values[i], 
                         Europe['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
for i in np.arange(len(N_America)):
    folium.Marker([N_America['Latitude'][i], N_America['Longitude'][i]], 
                  icon=folium.Icon(color='red',icon='rocket', prefix='fa'), 
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(N_America['Location'].values[i], 
                         N_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
for i in np.arange(len(S_America)):
    folium.Marker([S_America['Latitude'][i], S_America['Longitude'][i]], 
                  icon=folium.Icon(color='red',icon='rocket', prefix='fa'), 
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(S_America['Location'].values[i], 
                         S_America['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
for i in np.arange(len(Oceania)):
    folium.Marker([Oceania['Latitude'][i], Oceania['Longitude'][i]], 
                  icon=folium.Icon(color='green',icon='rocket', prefix='fa'), 
                  popup=folium.Popup('{0} \n Coordinates: {1}'.format(Oceania['Location'][i], 
                         Oceania['Coordinates'][i].split('\ufeff / ')[0]),parse_html=True)
                  ).add_to(Map_icon)
Map_icon

In [52]:
fn_out3 = "map_rocket_launch_sites_icon.html"
Map_cluster.save(fn_out3)